In [13]:
!pip install tensorflow
import pandas as pd
import numpy as np
import tensorflow as tf

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [20]:
ratings_df = pd.read_csv('./ml-latest-small/ratings.csv')
ratings_df.tail()
#tail命令用於輸入檔案中的尾部內容。tail命令預設在屏幕上顯示指定檔案的末尾5行。
movies_df = pd.read_csv('./ml-latest-small/movies.csv')
movies_df.tail()

movies_df['movieRow'] = movies_df.index
#生成一列‘movieRow’，等於索引值index
movies_df.tail()


movies_df = movies_df[['movieRow','movieId','title']]
#篩選三列出來
movies_df.to_csv('./ml-latest-small/moviesProcessed.csv', index=False, encoding='utf-8')
#生成一個新的檔案moviesProcessed.csv
movies_df.tail()


ratings_df = pd.merge(ratings_df, movies_df, on='movieId')
ratings_df.head()



ratings_df = ratings_df[['userId','movieRow','rating']]
#篩選出三列
ratings_df.to_csv('./ml-latest-small/ratingsProcessed.csv', index=False,  encoding='utf-8')
#匯出一個新的檔案ratingsProcessed.csv
ratings_df.head()

,userId,movieRow,rating
0,1,0,4.0
1,5,0,4.0
2,7,0,4.5
3,15,0,2.5
4,17,0,4.5


In [21]:
userNo = ratings_df['userId'].max() + 1
#userNo的最大值
movieNo = ratings_df['movieRow'].max() + 1
#movieNo的最大值

rating = np.zeros((movieNo,userNo))
#創建一個值都是0的資料
flag = 0
ratings_df_length = np.shape(ratings_df)[0]
#查看矩陣ratings_df的第一維度是多少
for index,row in ratings_df.iterrows():
    #interrows（），對錶格ratings_df進行遍歷
    rating[int(row['movieRow']),int(row['userId'])] = row['rating']
    #將ratings_df表裡的'movieRow'和'userId'列，填上row的‘評分’
    flag += 1

record = rating > 0
record
record = np.array(record, dtype = int)
#更改資料型別，0表示用戶沒有對電影評分，1表示用戶已經對電影評分
record

array([[0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
def normalizeRatings(rating, record):
    m, n =rating.shape
    #m代表電影數量，n代表用戶數量
    rating_mean = np.zeros((m,1))
    #每部電影的平均得分
    rating_norm = np.zeros((m,n))
    #處理過的評分
    for i in range(m):
        idx = record[i,:] !=0
        #每部電影的評分，[i，:]表示每一行的所有列
        rating_mean[i] = np.mean(rating[i,idx])
        #第i行，評過份idx的用戶的平均得分；
        #np.mean() 對所有元素求均值
        rating_norm[i,idx] -= rating_mean[i]
        #rating_norm = 原始得分-平均得分
    return rating_norm, rating_mean
rating_norm, rating_mean = normalizeRatings(rating, record)

rating_norm =np.nan_to_num(rating_norm)
#對值為NaNN進行處理，改成數值0
rating_norm

rating_mean =np.nan_to_num(rating_mean)
#對值為NaNN進行處理，改成數值0
rating_mean

array([[3.92093023],
       [3.43181818],
       [3.25961538],
       ...,
       [3.5       ],
       [3.5       ],
       [4.        ]])

In [27]:
num_features = 10
X_parameters = tf.Variable(tf.random.uniform([movieNo, num_features],seed = 0.35))
Theta_parameters = tf.Variable(tf.random.uniform([userNo, num_features],seed = 0.35))
#tf.Variables()初始化變數
#tf.random_normal()函式用於從服從指定正太分佈的數值中取出指定個數的值，mean: 正態分佈的均值。stddev: 正態分佈的標準差。dtype: 輸出的型別
loss = 1/2 * tf.reduce_sum(((tf.matmul(X_parameters, Theta_parameters, transpose_b = True) - rating_norm) * record) ** 2) + 1/2 * (tf.reduce_sum(X_parameters ** 2) + tf.reduce_sum(Theta_parameters ** 2))
#基於內容的推薦演算法模型


# 函式解釋：
# reduce_sum() 就是求和，reduce_sum( input_tensor, axis=None,  keep_dims=False, name=None, reduction_indices=None)
# reduce_sum() 引數解釋：
# 1) input_tensor：輸入的張量。
# 2) axis：沿著哪個維度求和。對於二維的input_tensor張量，0表示按列求和，1表示按行求和，[0, 1]表示先按列求和再按行求和。
# 3) keep_dims：預設值為Flase，表示預設要降維。若設為True，則不降維。
# 4) name：名字。
# 5) reduction_indices：預設值是None，即把input_tensor降到 0維，也就是一個數。對於2維input_tensor，reduction_indices=0時，按列；reduction_indices=1時，按行。
# 6) 註意，reduction_indices與axis不能同時設置。

# tf.matmul（a,b）,將矩陣 a 乘以矩陣 b，生成a * b
# tf.matmul（a,b）引數解釋：
# 1) a：型別為 float16，float32，float64，int32，complex64，complex128 和 rank > 1的張量。
# 2) b：與 a 具有相同型別和 rank。
# 3) transpose_a：如果 True，a 在乘法之前轉置。
# 4) transpose_b：如果 True，b 在乘法之前轉置。
# 5) adjoint_a：如果 True，a 在乘法之前共軛和轉置。
# 6) adjoint_b：如果 True，b 在乘法之前共軛和轉置。
# 7) a_is_sparse：如果 True，a 被視為稀疏矩陣。
# 8) b_is_sparse：如果 True，b 被視為稀疏矩陣。
# 9) name：操作名稱（可選）

In [40]:
optimizer =  tf.optimizers.Adam(1e-4)
# https://blog.csdn.net/lenbow/article/details/52218551


In [51]:
# Optimizer.minimize對一個損失變數基本上做兩件事
# 它計算相對於模型引數的損失梯度。
# 然後應用計算出的梯度來更新變數。


# https://blog.csdn.net/lenbow/article/details/52218551
train = optimizer.minimize(loss,var_list=rating_norm)

ValueError: Passed in object of type <class 'numpy.ndarray'>, not tf.Tensor